In [89]:
import pandas as pd
import ast
import numpy as np
import json

In [90]:
proj_git_summary = pd.read_csv('awesome_proj_data_github.csv').drop(columns = ['Unnamed: 0']).drop_duplicates(subset = 'Project', keep = 'last')
proj_git_summary.head()

,Project,Url,Number of watchers,Number of forks,Number of issues,Number of pull requests,Number of commits,Fork urls
0,MvvmCross,https://github.com/MvvmCross/MvvmCross/,272.0,1256,228.0,29.0,9185.0,"['https://github.com/MvvmCross/MvvmCross', 'ht..."
1,RawCMS,https://github.com/arduosoft/RawCMS/,12.0,58,32.0,5.0,745.0,"['https://github.com/arduosoft/RawCMS', 'https..."
2,Shouldly,https://github.com/shouldly/shouldly/,65.0,353,30.0,3.0,1211.0,"['https://github.com/shouldly/shouldly', 'http..."
3,Neovim,https://github.com/neovim/neovim/,1000.0,3042,1.0,304.0,16999.0,"['https://github.com/neovim/neovim', 'https://..."
4,grok.net,https://github.com/Marusyk/grok.net/,3.0,8,1.0,NaN,32.0,"['https://github.com/Marusyk/grok.net', 'https..."


In [91]:
proj_git_summary.shape

(208, 8)

In [92]:
# grab contributor data -- are there any similarities?
contributors = pd.read_csv('awesome_proj_data_contributors.csv').drop(columns = ['Unnamed: 0']).append(pd.read_csv('awesome_proj_data_contributors_rescrape.csv').drop(columns = ['Unnamed: 0'])).append(pd.read_csv('awesome_proj_data_contributors_rescrape2.csv').drop(columns = ['Unnamed: 0'])).drop_duplicates(subset = 'Project', keep = 'last')
contributors.head()

,Project,Contributors
0,MvvmCross,"['slodge', 'martijn00', 'Cheesebaron', 'nmilco..."
1,RawCMS,"['marcuson', 'zeppaman', 'keggyy', 'girardengo..."
2,Shouldly,"['JakeGinnivan', 'SimonCropp', 'JosephWoodward..."
3,Neovim,"['justinmk', 'janlazo', 'ZyX-I', 'tarruda', 'j..."
4,grok.net,"['Marusyk', 'Marusyk', 'Justin-Lloyd', 'iBlack..."


In [93]:
list(contributors.loc[contributors['Contributors'] == '[]']['Project'])

[]

In [94]:
contrib_list = [ast.literal_eval(i) for i in contributors['Contributors']]
contrib_list[0]

['slodge',
 'martijn00',
 'Cheesebaron',
 'nmilcoff',
 'nickrandolph',
 'MarcBruins',
 'kjeremy',
 'hollywoof',
 'Plac3hold3r',
 'tofutim',
 'mvanbeusekom',
 'lothrop',
 'heytherewill',
 'csteeg',
 'Garfield550',
 'thefex',
 'guillaume-fr',
 'vvolkgang',
 'tbalcom',
 'orzech85',
 'andres-gimenez',
 'jz5',
 'LRP-sgravel',
 'stoneman',
 'andyci',
 'spockfish',
 'danielcweber',
 'Dexyon',
 'biozal',
 'jamie94bc',
 'ehuna',
 'b099l3',
 'PeterBurke',
 'MartinZikmund',
 'steveydee82',
 'nschoenberg',
 'johnnywebb',
 'fedemkr',
 'Bowman74',
 'softlion',
 'felixfxu',
 'Stephanvs',
 'IlSocio',
 'AbdelrahmanGIT',
 'tritter',
 'prin53',
 'mellson',
 'markuspalme',
 'jamsoft',
 'dbeattie71',
 'gentledepp',
 'Prandtl',
 'MKuckert',
 'Laumania',
 'Giorgi',
 'DaRosenberg',
 'vatsalyagoel',
 'mgjhl',
 'ddunkin',
 'danzel',
 'Strifex',
 'ShawInnes',
 'RayMMond',
 'sdebruyn',
 'jimbobbennett',
 'geirsagberg',
 'PelleRavn',
 'Hackmodford',
 'AnthonyNjuguna',
 'tomcurran',
 'mattwhetton',
 'benschi11',
 '

In [95]:
count_i = 0
intersections = []
for i in contrib_list:
    count_j = 0
    proj_i = str(contributors.iloc[count_i]['Project'])
    for j in contrib_list[count_i + 1:]:
        proj_j = str(contributors.iloc[count_j]['Project'])
        intersect = list(set(i) & set(j))
        if len(intersect) > 0:
            intersections.append((proj_i, proj_j, intersect))
        count_j += 1
    count_i += 1

In [96]:
len(intersections)

1348

In [97]:
len(contrib_list) * (len(contrib_list) - 1)

43056

In [98]:
1348/43056

0.03130806391675957

In [99]:
intersections[0]

('MvvmCross', 'osu!', ['MartinZikmund'])

In [100]:
nodes = []
links = []

has_link = {}
no_link = []
for i in intersections:
    new_link = {
        'source': i[0],
        'target': i[1],
        'value': len(i[2])
    }
    has_link[i[0]] = 1
    has_link[i[1]] = 1
    links.append(new_link)
    
for index, row in proj_git_summary.iterrows():
    new_node_id = row['Project']
    new_node_watchers = 0 if np.isnan(row['Number of watchers']) else int(row['Number of watchers'])
    new_node_forks = int(row['Number of forks'])
    new_node_issues = 0 if np.isnan(row['Number of issues']) else int(row['Number of issues'])
    new_node_prs = 0 if np.isnan(row['Number of pull requests']) else int(row['Number of pull requests'])
    new_node_commits = 0 if np.isnan(row['Number of commits']) else int(row['Number of commits'])
    new_node = {
        'id': new_node_id,
        'watchers': new_node_watchers,
        'forks': new_node_forks,
        'issues': new_node_issues,
        'prs': new_node_prs,
        'commits': new_node_commits
    }
    nodes.append(new_node)
    
    # handle nodes with no intersections
    if new_node_id not in has_link:
        no_link.append(new_node_id)
    
print(nodes[:5])
print(links[:5])

[{'id': 'MvvmCross', 'watchers': 272, 'forks': 1256, 'issues': 228, 'prs': 29, 'commits': 9185}, {'id': 'RawCMS', 'watchers': 12, 'forks': 58, 'issues': 32, 'prs': 5, 'commits': 745}, {'id': 'Shouldly', 'watchers': 65, 'forks': 353, 'issues': 30, 'prs': 3, 'commits': 1211}, {'id': 'Neovim', 'watchers': 1000, 'forks': 3042, 'issues': 1, 'prs': 304, 'commits': 16999}, {'id': 'grok.net', 'watchers': 3, 'forks': 8, 'issues': 1, 'prs': 0, 'commits': 32}]
[{'source': 'MvvmCross', 'target': 'osu!', 'value': 1}, {'source': 'MvvmCross', 'target': 'Helm', 'value': 1}, {'source': 'MvvmCross', 'target': 'PouchDB', 'value': 1}, {'source': 'MvvmCross', 'target': 'Yarn', 'value': 1}, {'source': 'RawCMS', 'target': 'MvvmCross', 'value': 1}]


In [101]:
no_link

['SirixDB',
 'Symfony',
 'PrestaShop',
 'pandas',
 'ArviZ',
 'PublicLab.org',
 'open-build-service',
 'Matestack',
 'TypeScript',
 'reatom',
 'Booster']

In [102]:
# create min distance edges between each pair in no_link
count = 0
for i in no_link:
    for j in no_link[count + 1:]:
        new_link = {
            'source': i,
            'target': j,
            'value': 0
        }
        links.append(new_link)
        print(new_link)
    count += 1

{'source': 'SirixDB', 'target': 'Symfony', 'value': 0}
{'source': 'SirixDB', 'target': 'PrestaShop', 'value': 0}
{'source': 'SirixDB', 'target': 'pandas', 'value': 0}
{'source': 'SirixDB', 'target': 'ArviZ', 'value': 0}
{'source': 'SirixDB', 'target': 'PublicLab.org', 'value': 0}
{'source': 'SirixDB', 'target': 'open-build-service', 'value': 0}
{'source': 'SirixDB', 'target': 'Matestack', 'value': 0}
{'source': 'SirixDB', 'target': 'TypeScript', 'value': 0}
{'source': 'SirixDB', 'target': 'reatom', 'value': 0}
{'source': 'SirixDB', 'target': 'Booster', 'value': 0}
{'source': 'Symfony', 'target': 'PrestaShop', 'value': 0}
{'source': 'Symfony', 'target': 'pandas', 'value': 0}
{'source': 'Symfony', 'target': 'ArviZ', 'value': 0}
{'source': 'Symfony', 'target': 'PublicLab.org', 'value': 0}
{'source': 'Symfony', 'target': 'open-build-service', 'value': 0}
{'source': 'Symfony', 'target': 'Matestack', 'value': 0}
{'source': 'Symfony', 'target': 'TypeScript', 'value': 0}
{'source': 'Symfony', 

In [103]:
data = {
    'nodes': nodes,
    'links': links
}

with open('summary_git_data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii = False, indent = 4)